In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split

In [2]:
def prepare_data(train_data, train_label, memory_unit):
    number, height, width, channel = \
    train_data.shape[0], train_data.shape[1], train_data.shape[2], train_data.shape[3]

    number_of_mem_unit = number // memory_unit

    # let the data become from (1200, 80, 160, 1) to (40, 30, 80 * 160)
    sample_number = number_of_mem_unit * memory_unit
    unit_train_data = train_data[0: sample_number, :, :, :].reshape(-1, memory_unit, height * width)
    
    # label shift to right by one step
    unit_train_label = train_label[1: sample_number + 1, :, :, :].reshape(-1, memory_unit, height * width)
    
    return iter(unit_train_data), iter(unit_train_label)


In [3]:
# Example of formatting
labels = pickle.load(open("full_CNN_labels.p", "rb" ))
labels = np.array(labels)[0:12000,:,:,:]
labels = labels / 255
print(labels.shape)
print(labels.reshape(-1, 30, 80 * 160).shape)

(12000, 80, 160, 1)
(400, 30, 12800)


In [4]:
data = np.array(pickle.load(open("full_CNN_labels.p", "rb" )))
label = np.array(pickle.load(open("full_CNN_labels.p", "rb" )))
label = label / 255

train_ratio = 0.8
memory_size = 30
train_data, test_data, train_label, test_label = \
train_test_split(data, label, train_size = train_ratio,
                              test_size = 1 - train_ratio,
                              shuffle = False)

input_data_iter, input_label_iter = prepare_data(train_data, train_label, memory_size)


In [ ]:
#Cell used to training

learning_rate = 0.001

#Size of our input image. e.g. number of features
number_of_features = 80 * 160

steps_num = 200

#input data
X = tf.placeholder(tf.float32, [None, memory_size, number_of_features])
#data label
Y = tf.placeholder(tf.float32, [None, memory_size, number_of_features])

#RNN cells
cell = tf.contrib.rnn.BasicLSTMCell(number_of_features, activation=tf.nn.relu)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

#loss
loss = tf.reduce_mean(tf.square(outputs - Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for iteration in range(steps_num):
        train_data, train_labels = next(input_data_iter), next(input_label_iter)
        sess.run(train, feed_dict = {X:train_data, y:train_labels})
        error = loss.eval( feed_dict = {X:train_data, y:train_labels})
        print("iteration {}, error is {}".format(iteration, error))
        print("\n")
    saver.save("./tmp/rnn_model")

In [ ]:
# Cell used to predicting
with tf.Session() as sess:
    saver.restore(sess, "./tmp/rnn_model")
    # input data used to predict, format:
    x_input = ??????
    # The output should be a 1-D array whose size is height * width. 
    # In order to display it, it needs reshpe
    y_pred = sess.run(outputs, feed_dict={X:x_input})